In [29]:
import requests
import pandas as pd
import urllib.parse
import json
from datetime import datetime
from pprint import pprint

class ssc:

    def __init__(self,id=''):

        self.id = id

    #Be able to get data for a specific table []
    #Be able to get data for all tables in a specific category []
    #Be able to get data for all tables []
    #Be able to browse or search tables by ID, name, category

    def _table_info(id='',lang='en'):

        url = "https://www150.statcan.gc.ca/t1/wds/rest/getAllCubesList"
        resp = requests.get(url)
        table = resp.json()

        for i in table:
            if i['productId'] == id: return i
        else:
            print(f'Table {id} not found.')
        
    def _parse_dim(x):

        tmp = {}
        sel = []

        for i in x:

            tmp[i['dimensionNameEn']] = i['hasUOM']
            sel.append([1])
        
        return tmp,sel
    
    def get_metadata(id, timeout=30, lang='all'):
        """
        Retrieves metadata for a cube (table) from Statistics Canada WDS.

        Args:
            id (int or str): The product ID (PID) of the cube (e.g., 35100003).
            timeout (float): Timeout in seconds for the HTTP request.
            lang (str): Language preference for the metadata ('en', 'fr', or 'all').

        Returns:
            dict: JSON response object converted to Python dict.
        """

        endpoint = "https://www150.statcan.gc.ca/t1/wds/rest/getCubeMetadata"

        # Prepare payload as list of dicts, per documentation example. :contentReference[oaicite:2]{index=2}
        payload = [
            {
                "productId": str(id)
            }
        ]

        headers = {
            "Content-Type": "application/json"
        }

        def _remove_lang(obj,language):
            
            if isinstance(obj, dict):
                
                obj = {k: _remove_lang(v,language) for k, v in obj.items() if not k.endswith(language)}

                for k in obj.keys():
                    if isinstance(obj[k],list):
                        if len(obj[k]) > 0:
                            if isinstance(obj[k][0],dict):
                                print(k)
                                obj[k] = [_remove_lang(i,language) for i in obj[k]]
                return obj
            
            else:
                return obj

        try:
            response = requests.post(endpoint, headers=headers, json=payload, timeout=timeout)
            response.raise_for_status()
            data = response.json()

            # Check for status
            if data[0]["status"] != "SUCCESS":
                raise RuntimeError(f"Request returned non-SUCCESS status: {data}")
            
            if lang == 'en':
                return _remove_lang(data[0]['object'],'Fr')
            
            elif lang == 'fr':
                return _remove_lang(data[0]['object'],'En')
            
            else:
                return data[0]['object']
        
        except requests.RequestException as e:
            raise RuntimeError(f"HTTP request failed: {e}") from e
        
        except json.JSONDecodeError as e:
            raise RuntimeError(f"Failed to parse JSON response: {e}") from e

    def get_table(id='',periods=1,full=True,enc='%2C1%2C2',region_type='',lang='en'):

        p = ssc._table_info(id)

        tablename = p['cubeTitleEn']
        dim = p['dimensions']
        dim,selected = ssc._parse_dim(dim)

        raw = json.dumps(selected, separators=(',',':'))
        enc = urllib.parse.quote(raw)

        url = f'https://www150.statcan.gc.ca/t1/tbl1/en/dtl!downloadDbLoadingData-nonTraduit.action?pid={id}01&latestN={periods}&startDate=&endDate=&csvLocale=en&selectedMembers={enc}&checkedLevels=0D1%2C0D1%2C0D2%2C0D3'
        
        if full: print('Warning: Unfiltered tables can be very large')
        print(url)
        
        df = pd.read_csv(url)
        if region_type != '':
            df = df[df.DGUID.str[6:9] == region_type]

        display(df)

    def list_tables(lang='en'):

        url = "https://www150.statcan.gc.ca/t1/wds/rest/getAllCubesList"
        resp = requests.get(url)
        tables = resp.json()

        df = pd.DataFrame(tables)

        if lang == 'en':
            df = df[[x for x in df.columns if x if 'Fr' not in x]]
        elif lang == 'fr':
            df = df[[x for x in df.columns if x if 'En' not in x]]

        df = df[df['archived'] == '2']

        archived_tables = df[df['archived'] == '1']
        active_tables = df[df['archived'] == '2']

        card_css = "border:1px solid black;margin:10px;padding:10px;background:white;max-width:200px;border-radius:3px;color:black;"
        n = 2

        count = 0

        for p in tables:

            id = p['productId']
            endDate = datetime.fromisoformat(p['cubeEndDate'].replace("Z", "+00:00")).replace(tzinfo=None)

            if p['archived'] == '2':

                metadata = get_metadata(id)
                num_dimensions = len(metadata['dimension'])
                data = metadata['dimension']

                tablename = p['cubeTitleEn']
                dim = p['dimensions']
                dim,selected = ssc._parse_dim(dim)

                raw = json.dumps(selected, separators=(',',':'))
                enc = urllib.parse.quote(raw)

                num_param = len(data)

                attributes = {}
                attributes['Table Name'] = tablename

                for i in range(0,num_param):

                    sub_size = len(data[i]['member'])
                    top_name = data[i]['dimensionNameEn']

                    for j in range(0,sub_size):
                        
                        #print(data[i]['member'][j])

                        name = data[i]['member'][j]['memberNameEn']
                        classification = data[i]['member'][j]['classificationCode']

                        if top_name in attributes:
                            attributes[top_name].append(name)
                        else:
                            attributes[top_name] = [name]

                print(id,' - ',tablename,'\n',attributes)
                #get_table(id=id,n=n,enc=enc)#region_type='503')
                
                count += 1

                if count > 5:
                    break
                    

#meta = ssc.get_metadata(id=35100003,lang='en')
#pprint(meta)
df = ssc.get_table(id=35100003)



https://www150.statcan.gc.ca/t1/tbl1/en/dtl!downloadDbLoadingData-nonTraduit.action?pid=3510000301&latestN=1&startDate=&endDate=&csvLocale=en&selectedMembers=%5B%5B1%5D%2C%5B1%5D%5D&checkedLevels=0D1%2C0D1%2C0D2%2C0D3


,REF_DATE,GEO,DGUID,Custodial and community supervision,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2023/2024,Provinces and Territories,2021A000011124,Total actual-in count,Persons,249,units,0,v65927052,15.1,614.1,NaN,NaN,NaN,1
1,2023/2024,Provinces and Territories,2021A000011124,Total actual-in count,Persons,249,units,0,v65927052,15.1,614.1,NaN,NaN,NaN,1
2,2023/2024,Newfoundland and Labrador,2021A000210,Total actual-in count,Persons,249,units,0,v32164121,1.1,NaN,..,NaN,NaN,1
3,2023/2024,Newfoundland and Labrador,2021A000210,Total actual-in count,Persons,249,units,0,v32164121,1.1,NaN,..,NaN,NaN,1
4,2023/2024,Prince Edward Island,2021A000211,Total actual-in count,Persons,249,units,0,v32164133,2.1,2.7,NaN,NaN,NaN,1
5,2023/2024,Nova Scotia,2021A000212,Total actual-in count,Persons,249,units,0,v32164145,3.1,12.0,NaN,NaN,NaN,1
6,2023/2024,New Brunswick,2021A000213,Total actual-in count,Persons,249,units,0,v32164154,4.1,7.9,NaN,NaN,NaN,1
7,2023/2024,Quebec,2021A000224,Total actual-in count,Persons,249,units,0,v32164165,5.1,NaN,..,NaN,NaN,1
8,2023/2024,Ontario,2021A000235,Total actual-in count,Persons,249,units,0,v32164177,6.1,312.4,NaN,NaN,NaN,1
9,2023/2024,Manitoba,2021A000246,Total actual-in count,Persons,249,units,0,v32164188,7.1,110.2,NaN,NaN,NaN,1
